# 트랜스포머를 이용한 한국어 챗봇

reference : https://wikidocs.net/89786

## 1. 데이터 로드

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")

('ChatBotData.csv', <http.client.HTTPMessage at 0x207d76df1f0>)

In [3]:
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
print('챗봇 샘플의 개수 :', len(train_data))

챗봇 샘플의 개수 : 11823


In [5]:
# Null값 확인
# .isnull()은 DataFrame의 요소가 null인 경우 True를, 아니면 False를 반환한다. 그것 sum했는데 0이 나왔으므로 Null값은 없다.
print(train_data.isnull().sum())

Q        0
A        0
label    0
dtype: int64


? . ! 과 같은 구두점 앞에 공백을 추가하여 다른 문자들과 구분한다.

In [6]:
# 질문 데이터에 구두점 전처리 적용
questions = []
for sentence in train_data['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    """
    - r"([?.!,])" 는 ?.!, 중하나를 찾는다
    - r" \1 " 는 찾은 구두점(?.!,)의 양쪽에 공백을 추가한다. 
    - sentence.strip()은 문장의 앞뒤 공백을 제거한다.
    """
    questions.append(sentence)

In [7]:
# 답변 데이터에 구두점 전처리 적용
answers = []
for sentence in train_data['A']:
    # 구두점에 대해서 띄어쓰기
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [8]:
# 구두점들이 분리되었는지 확인
print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


## 2. 단어 집합 생성

In [9]:
# 서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어 집합(vocabulary) 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13
)

"""
- questions와 answers 리스트를 합쳐서 하나의 큰 리스트로 만들고 이것이 build_from_corpus의 입력으로 사용된다.
- 단어 집합 크기를 2^13(8192)로 설정한다.
"""

'\n- questions와 answers 리스트를 합쳐서 하나의 큰 리스트로 만들고 이것이 build_from_corpus의 입력으로 사용된다.\n- 단어 집합 크기를 2^13(8192)로 설정한다.\n'

In [10]:
tokenizer.vocab_size  # 토큰이 총 8187개 라는 것을 의미한다.

8178

시작 토큰 SOS와 종료 토큰 EOS 존재한다. 이것들도 단어 집합에 포함시켜야 한다.

In [11]:
# 시작 토큰과 종료 토큰에 대한 정수 부여
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]  # [8178], [8179] 기존에는 0부터 8186까지의 정수 인덱스

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

패딩에 사용될 0번 토큰부터, 마지막 토큰인 8179토큰까지의 개수 카운트 - 단어 집합의 크기는 8,180개

In [12]:
print('시작 토큰 번호 :', START_TOKEN)
print('종료 토큰 번호 :', END_TOKEN)
print('단어 집합의 크기 :', VOCAB_SIZE)

시작 토큰 번호 : [8178]
종료 토큰 번호 : [8179]
단어 집합의 크기 : 8180


## 3. 정수 인코딩과 패딩

In [13]:
# 서브워드텍스트인코더 토크나이저의 .encode()를 사용하여 텍스트 시퀀스를 정수 시퀀스로 변환.
print('임의의 질문 샘플을 정수 인코딩 : {}'.format(tokenizer.encode(questions[20])))

임의의 질문 샘플을 정수 인코딩 : [5766, 611, 3509, 141, 685, 3747, 849]


반대로 정수 인코딩 된 결과는 다시 decode()를 사용하여 기존의 텍스트 시퀀스로 복원할 수 있다.

In [14]:
# 서브워드텍스트인코더 토크나이저의 .encode()와 .decode() 테스트해보기
# 임의의 입력 문장을 sample_string에 저장
sample_string = questions[20]

# encode() : 텍스트 시퀀스 --> 정수 시퀀스
tokenized_string = tokenizer.encode(sample_string)
print('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# decode() : 정수 시퀀스 --> 텍스트 시퀀스
original_string = tokenizer.decode(tokenized_string)
print('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [5766, 611, 3509, 141, 685, 3747, 849]
기존 문장: 가스비 비싼데 감기 걸리겠어


정수는 7개인데 어절은 4개이다. 왜? '가스비' 나 '비싼데'라는 한 어절이 정수 인코딩 후에는 두 개 이상의 정수일 수 있다는 것이다.  
각 정수가 어떤 서브워드로 맵핑되는지 출력해보자

In [15]:
# 각 정수는 각 단어와 어떻게 mapping되는지 병렬로 출력
# 서브워드텍스트인코더는 의미있는 단위의 서브워드로 토크나이징한다. 띄어쓰기 단위 X 형태소 분석 단위 X
for ts in tokenized_string:
    print('{}---->{}'.format(ts, tokenizer.decode([ts])))

5766---->가스
611---->비 
3509---->비싼
141---->데 
685---->감기 
3747---->걸리
849---->겠어


이제 전체 데이터에 대해서 정수 인코딩과 패딩을 진행한다.  

이를 위한 함수로 tokenize_and_filter()를 만든다. 임의로 패딩의 길이는 40으로 정했다.

In [16]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 40

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outpus):
    """
        - inputs (List[str]): questions에 해당하는 질문들
        - outpus (List[str]): answers에 해당하는 답변들
    """
    tokenized_inputs, tokenized_outputs = [], []
    
    for (sentence1, sentence2) in zip(inputs, outpus):
        # encode(텍스트 시퀀스를 정수 시퀀스로 변환), 시작 토큰과 종료 토큰 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN  
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
        
        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)
        
    # 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post'
    ) # pad_sequence는 기본적으로 앞에 0을 채우므로, 뒤에 0을 채우고 싶다면 padding='post'를 해줘야 한다.
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post'
    )
    
    return tokenized_inputs, tokenized_outputs

In [17]:
questions, answers = tokenize_and_filter(questions, answers)

In [18]:
print('질문 데이터의 크기(shape) :', questions.shape)
print('답변 데이터의 크기(shape) :', answers.shape)

질문 데이터의 크기(shape) : (11823, 40)
답변 데이터의 크기(shape) : (11823, 40)


In [19]:
# 0번 샘플을 임의로 출력
print(questions[0])
print(answers[0])

[8178 7915 4207 3060   41 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


## 인코더와 디코더의 입력, 그리고 레이블 만들기

tf.data.Dataset을 사용하여 데이터를 배치 단위로 불러올 수 있다.

In [20]:
# tensorflow dataset을 이용하여 셔플을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한, 이 과정에서 교사 강요를 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.

BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
# 교사 강요를 쓰므로 진짜 정답 시퀀스를 디코더의 입력으로 사용한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,  # questions를 인코더의 입력 데이터로 설정
        'dec_inputs': answers[:, :-1]  # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
        # 디코더는 현재 단어를 보고 다음 단어를 예측하므로 마지막 단어를 받으면 그 다음에 올 단어가 없다
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 즉, 시작 토큰이 제거된다
        # 시작 토큰은 예측할 필요가 없기 때문, 실제 출력은 시작 토큰 없이 시작되므로 디코더의 출력 시퀀스에서 시작 토큰 제거
    },
))
# 즉, 디코더의 입력은 START_TOKEN으로 시작해서 마지막 단어 전 까지이며, 디코더의 출력은 첫 번째 실제 단어부터 마지막 단어까지이다.

dataset = dataset.cache()  # cache : 데이터셋의 데이터를 캐시에 저장하여 데이터 로딩 속도를 향상
dataset = dataset.shuffle(BUFFER_SIZE)  
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)  # 데이터 로딩속도를 향상시키기 위해서 미리 데이터를 가져오는것?

In [21]:
# 임의의 샘플에 대해서 [:, :-1]과 [:, 1:]이 어떤 의미를 가지는지 테스트해본다.
print(answers[0]) # 기존 샘플
print(answers[:1][:, :-1])  # 마지막 패딩 토큰 제거, 길이 39
print(answers[:1][:, 1:])  # 시작 토큰 제거, 길이 39

[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[[8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
[[3844   74 7894    1 8179    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]


## 5. 트랜스포머 만들기

In [22]:
# 이전에 만들었던 트랜스포머 이용. Transformer_Tensorflow.py 에 있다.
from Transformer_Tensorflow import transformer, CustomSchedule

In [23]:
tf.keras.backend.clear_session()  # 현재 세션에 존재하는 모든 텐서 객체들을 삭제. 

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    max_length=MAX_LENGTH,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT
    )

In [24]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape = (-1, MAX_LENGTH - 1))
    # y_true의 shape와 y_pred의 shape가 일치하도록 조정하는 것 같다.
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)
    
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    # 실제 단어가 아니라 패딩으로 채워진 부분들이 있다. 이 부분을 loss function 계산에서 제외
    # 0이 아니면 True를 반환. 즉 0인 부분들은 전부 0.0이 되고 0이 아니면 1.0이 된다
    loss = tf.multiply(loss, mask)
    # 패딩된 부분의 loss를 0으로 만든다
    
    return tf.reduce_mean(loss)

In [25]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
  learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape = (-1, MAX_LENGTH - 1))
  
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [26]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
185/185 [==============================] - 24s 100ms/step - loss: 1.4499 - accuracy: 0.0291
Epoch 2/50
185/185 [==============================] - 18s 98ms/step - loss: 1.1745 - accuracy: 0.0495
Epoch 3/50
185/185 [==============================] - 18s 97ms/step - loss: 1.0038 - accuracy: 0.0508
Epoch 4/50
185/185 [==============================] - 18s 98ms/step - loss: 0.9295 - accuracy: 0.0544
Epoch 5/50
185/185 [==============================] - 18s 97ms/step - loss: 0.8718 - accuracy: 0.0574
Epoch 6/50
185/185 [==============================] - 18s 98ms/step - loss: 0.8130 - accuracy: 0.0613
Epoch 7/50
185/185 [==============================] - 18s 98ms/step - loss: 0.7488 - accuracy: 0.0671
Epoch 8/50
185/185 [==============================] - 18s 99ms/step - loss: 0.6770 - accuracy: 0.0747
Epoch 9/50
185/185 [==============================] - 18s 98ms/step - loss: 0.5980 - accuracy: 0.0835
Epoch 10/50
185/185 [==============================] - 18s 98ms/step - loss: 0.51

## 6. 챗봇 평가하기

In [27]:
def preprocess_sentence(sentence):
    # 단어와 구두점 사이에 공백 추가
    # ex) 12시 떙! -> 12시 떙 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

In [28]:
def evaluate(sentence):
    
    # 입력 문장에 대한 전처리
    sentence = preprocess_sentence(sentence)
    
    # 입력 문장에 시작 토큰과 종료 토큰을 추가
    sentence = tf.expand_dims(
        START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0
    ) # tf.expand_dims(..., axis=0)은 [1,2,3]이 [[1,2,3]] 이렇게 되는 것이다.
      # 배치 사이즈에 대한 차원 추가인듯
    
    output = tf.expand_dims(START_TOKEN, 0) # 시작 토큰만으로 구성된 초기 디코더의 입력
    # 처음에는 디코더에게 시작 토큰만을 준다. 그리고 밑의 for문에 들어가게 된다.
    
    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)
        
        # 현재 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        # -1이면 가장 마지막꺼 이므로, 가장 최근의 예측값을 가져온다.
        
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        # 확률이 가장 높은 단어의 인덱스를 가져온다
        
        
        # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if tf.equal(predicted_id, END_TOKEN[0]):
            break
        
        # 현재 시점의 예측 단어를 output(출력)에 연결
        # output은 for문의 다음 루프에서 디코더의 입력이 된다.
        output = tf.concat([output, predicted_id], axis=-1)
        
    # 단어 예측이 모두 끝났다면 output을 리턴
    return tf.squeeze(output, axis = 0)

In [29]:
def predict(sentence):
    prediction = evaluate(sentence)
    
    # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
    # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩한다.
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size]
    )
    
    # print("Input : {}".format(sentence))
    # print("Output : {}".format(predicted_sentence))
    
    return predicted_sentence

In [30]:
predict("안녕 너 이름이 뭐야")

'위로봇이요 .'

In [31]:
predict("뭔 위로봇이여 너 김승희가 만든 이름 없는 챗봇이잖아")

'맘 고생 많았어요 .'

In [32]:
predict("넌 내가 만든 인공지능이다. 사람이 될 수 없어. 가만히 내가 말한거에 답이나 해")

'좋아하는 마음과 상관없이 좋아할 거예요 .'

In [33]:
predict("내가 너에 대한 존중이 없었다. 미안하다.")

'그런 사람 못 고쳐 씁니다 .'

In [34]:
predict("어떻게 알았지? 너 뭐냐? 내가 거짓말 하는거 어떻게 알았냐")

'그래도 참 씬스는 어때요 .'

In [35]:
predict("이제 너랑 안 놀고 게임할꺼다")

'또 다른 사랑이 찾아올 거예요 .'